# Visualizing and reducing WFM imaging data with Scipp

### Owen Arnold$^{1, 2}$, Neil Vaytet$^3$, Peter Kadletz$^4$, David Fairbrother$^1$

$^1$ STFC, Rutherford Appleton Laboratory, UK\
$^2$ Tessella, UK\
$^3$ Data Management and Software Centre, European Spallation Source, Copenhagen, Denmark\
$^4$ European Spallation Source ERIC, Lund, Sweden

*29 January 2021*

This notebook aims to demonstrate the capabilities of the Scipp software to visualize and reduce neutron imaging data in an intuitive way, via a python interface that is reminiscent of Numpy.

You can find this notebook at [this address](https://scipp.github.io/ess-notebooks/demos/imaging/demo-bragg-edge-imaging-2D.html).

#### Disclaimer

While we attempt to illustrate a realistic reduction workflow, scientific accuracy or quality are not the primary goals of this demo, and some steps have deliberately been simplified.
The aim is more to demonstrate what Scipp can do and what it is like to work with neutron data in a way that is natural and 'pythonic'.

## Experimental Summary

This script has been developed to measure local strain $\varepsilon$ defined as $\varepsilon = \Delta L/L_{0}$ in a FCC steel sample under elastic strain in a stress rig.
The measurements were measured at V20, HZB, Berlin, in September 2018 by Peter Kadletz.

$\lambda = 2 d \sin\theta$, where $2\theta = \pi$ (transmission), edges characterise the Bragg condition and hence $\lambda = 2 d$.
Therefore strain is easily computed from the wavelength measurement of a Bragg edge directly, using un-loaded vs loaded experimental runs (and reference mesurements).

The known Miller indices of the crystal structure (FCC) are used to predict the wavelength of the Bragg edges, which is bound by the reachable wavelength extents of the instrument.
This provides an approximate region to apply a fit.
A complement error function is used to fit each Bragg edge, and a refined centre location ($\lambda$) for the edge is used in the strain measurement.
Because each Bragg edge can be identified individually, one can determine anisotropic strain across the unit cell in the reachable crystallographic directions.

<img src="IMG_2290.jpg" width="500" />

# Setup

In [ ]:
import scipp as sc
import numpy as np
from scipp.plot import plot
import matplotlib.pyplot as plt
import utils
from utils import wfm

# Reduction

## Load the data files and instrument geometry

In [ ]:
ds = utils.load()

In [ ]:
ds

## Raw data visualization

In [ ]:
plot(ds["sample"])

## Converting time coordinate to TOF

Use the instrument geometry and chopper cascade parameters to compute time-distance diagram.

In [ ]:
plt.ion()
v20setup = wfm.v20.setup()
# Feed in the detector position to the instrument chopper cascade
v20setup['info']['detector_positions'] = {
    "GP2": -ds.coords['source-position'].value[2] + v20setup['info']['wfm_choppers_midpoint'] + sc.geometry.z(
    ds.coords["position"])['x', 0]['y', 0].value}

frames = wfm.get_frames(instrument=v20setup, plot=True)
frames

We overlay the frame locations onto a spectrum of integrated counts

In [ ]:
plt.ioff()
fig1, ax1 = plt.subplots()
plot(sc.sum(sc.sum(ds["reference"], 'x'), 'y'), ax=ax1)
for i in range(len(frames["GP2"]["left_edges"])):
    ax1.axvspan(frames["GP2"]["left_edges"][i], frames["GP2"]["right_edges"][i], color="C{}".format(i), alpha=0.3)
fig1.canvas.draw_idle()
fig1.canvas

Extract the sections in the original data using value-based slicing and shift the coordinates:

In [ ]:
plt.ion()
stitched = wfm.stitch(data=ds, dim="t", frames=frames["GP2"], plot=True)
stitched

To stitch the data, we make a common container with a TOF axis spanning the entire range, and sum the counts from the different frames.

In [ ]:
plt.ioff()
plot(sc.sum(sc.sum(stitched, 'x'), 'y'))

## Crop to relevant Tof section

We take a subset of the Tof range which contains the Bragg edges of interest:

In [ ]:
tof_start = 9.0e3*sc.units.us
tof_end = 2.75e4*sc.units.us
stitched = stitched["tof", tof_start:tof_end].copy()
plot(sc.sum(sc.sum(stitched, 'x'), 'y'))

## Transmission Masking

Divides the integrated sample counts with an open beam reference. Any values > masking threshold will be masked. The adj pixels step checks for random pixels which were left unmasked or masked with all their neighbours having the same mask value. These are forced to True or false depending on their neighbour value.

In [ ]:
integrated = sc.sum(stitched, 'tof')
integrated /= integrated["reference"]

In [ ]:
masking_threshold = 0.80 * sc.units.one

for key in ["sample", "sample_elastic"]:
    mask = integrated[key].data > masking_threshold
    # Apply some neighbour smoothing to the masks
    mask = utils.operations.mask_from_adj_pixels(mask)
    stitched[key].masks["non-sample-region"] = mask
stitched

In [ ]:
plot(sc.sum(stitched["sample"], "tof"))

## Convert to wavelength

Scipp's `neutron` submodule contains utilities specific to neutron science, and in particular unit conversions.

In [ ]:
wavelength = sc.neutron.convert(stitched, origin="tof", target="wavelength")

# Rebin to common wavelength axis
edges = sc.array(dims=["wavelength"],
                       values=np.linspace(2.0, 5.5, 129), unit=sc.units.angstrom)
wavelength = sc.rebin(wavelength, "wavelength", edges)

wavelength

## Apply mean filter

In [ ]:
for key in wavelength:
    wavelength[key].data = utils.operations.mean_from_adj_pixels(wavelength[key].data)
plot(wavelength["sample"])

## Other visualizations

### Show difference between sample and elastic

In [ ]:
plot(wavelength["sample"] - wavelength["sample_elastic"], vmin=-0.002, vmax=0.002, cmap="RdBu")

## Group detector pixels to increase signal to noise

In [ ]:
nbins=27
grouped = utils.groupby2D(wavelength, nbins=nbins)
for key, item in grouped.items():
    item.masks["zero-counts"] = item.data == 0.0*sc.units.counts

In [ ]:
plot(grouped["sample"])

## Normalization

In [ ]:
# Normalize by open beam
normalized = grouped / grouped["reference"]
del normalized["reference"]

In [ ]:
replacement = sc.Variable(value=0.0, variance=0.0)
kwargs = {"nan": replacement, "posinf": replacement, "neginf": replacement}
for k in normalized.keys():
    sc.nan_to_num(normalized[k].data, out=normalized[k].data, **kwargs)

plot(sc.sum(sc.sum(normalized, 'x'), 'y'), grid=True, title='I/I0')

# Bragg edge fitting

We will first carry out Bragg-edge fitting on the sum of all counts in the sample.

In [ ]:
sample_summed = sc.sum(sc.sum(normalized["sample"], 'x'), 'y')

## Calculate expected Bragg edge positions 

In [ ]:
# Bragg edge position, in Angstrom, taken from COD entry 9008469
FCC_a = 3.5

# These miller indices for the given unit cell yield bragg edges
# between the maximum and minimum wavelength range.
indices_FCC = [(1, 1, 1), (2, 0, 0), (2, 2, 0), (3, 1, 1)]

Bragg_edges_FCC = utils.create_Braggedge_list(FCC_a, indices_FCC)
sc.table(Bragg_edges_FCC)

## Run the fitting (currently using Mantid under the hood)

In [ ]:
fit_params = utils.fit(Bragg_edges_FCC, reference=sample_summed)

In [ ]:
import scipy as sp
fig, ax = plt.subplots()
sample_summed.plot(ax=ax)
for i, f in enumerate(fit_params):
    x = np.linspace(f['f1.x0'].value - 0.3, f['f1.x0'].value + 0.3, 100)
    y=f['f1.h'].value*sp.special.erf(f['f1.a'].value*(x-f['f1.x0'].value)) + (f['f0.A0'].value + f['f0.A1'].value*x)
    ax.plot(x, y, color="C{}".format(i+1), label=Bragg_edges_FCC.coords['miller-indices'].values[i])
ax.legend()
fig.canvas

## Create a strain map

We can create a strain map by fitting the Bragg edges inside each tile of the image.

In [ ]:
# Make an empty container to receive the strain values
strain_map = normalized["sample_elastic"]["wavelength", 0] * (0.0 * sc.units.one)

iprog = 0
step = 5

# Loop over all the tiles
for j in range(nbins):
    for i in range(nbins):

        prog = int(100 * (i + j*nbins) / (nbins*nbins))
        if prog % step == 0 and prog == iprog:
            print(prog, "% complete")
            iprog += step
        
        if sc.greater(sc.sum(normalized["sample_elastic"]['y', j]['x', i], "wavelength").data,
                      0.0*sc.units.one).value:
            elastic_fit = utils.fit(Bragg_edges_FCC,
                                    reference=normalized["sample_elastic"]['y', j]['x', i], quiet=True)
            strain_map['y', j]['x', i] = 0.5 * (elastic_fit[0]['f1.x0'] - fit_params[0]['f1.x0'])

In [ ]:
plot(strain_map, vmin=-0.02, vmax=0.02, cmap="RdBu", title="Lattice strain $\epsilon$ (1, 1, 1)")

### 3D visualization

In [ ]:
resampled = sc.rebin(wavelength["sample"], "x", sc.array(
    dims=['x'], values=np.linspace(-0.011, 0.011, 129), unit=sc.units.m))
resampled = sc.rebin(resampled, "y", sc.array(
    dims=['y'], values=np.linspace(-0.011, 0.011, 129), unit=sc.units.m))
plot(resampled, projection="3d") # thickness: 0.00001